In [58]:
Block = Tuple{UnitRange{Int64},UnitRange{Int64},UnitRange{Int64}}

function commands(filename::String; init_only::Bool=false)::Vector{Tuple{Bool, Block}}
    comms = Vector{Tuple{Bool, Block}}([])
    for line in readlines(filename)
        switch = startswith(line, "on")
        xrange = parse.(Int64, split(line[findfirst("x=", line)[2]+1:findfirst("y=", line)[1]-2],".."))
        yrange = parse.(Int64, split(line[findfirst("y=", line)[2]+1:findfirst("z=", line)[1]-2],".."))
        zrange = parse.(Int64, split(line[findfirst("z=", line)[2]+1:end],".."))
        if init_only && (any(abs.(xrange) .> 50) | any(abs.(yrange) .> 50) | any(abs.(zrange) .> 50))
            continue
        else
            push!(comms, (switch, (xrange[1]:xrange[2], yrange[1]:yrange[2], zrange[1]:zrange[2])))
        end
    end
    return comms
end

function block_intersect(block1::Block, block2::Block)::Block
    # Could short circuit this I suppose
    return Tuple(intersect(r1, r2) for (r1, r2) in zip(block1, block2))
end

function block_size(block::Block)::Int64
    return prod(length.(block))
end

function block_difference(block1::Block, block2::Block)::Vector{Block}
    out = Vector{Block}([
    # blocks
        (block1[1].start:block2[1].start-1, block1[2], block1[3]),
        (block2[1].stop+1:block1[1].stop, block1[2], block1[3]),
    # slabs
        (block2[1], block1[2].start:block2[2].start-1, block1[3]),
        (block2[1], block2[2].stop+1:block1[2].stop, block1[3]),
    # columns
        (block2[1], block2[2], block1[3].start:block2[3].start-1),
        (block2[1], block2[2], block2[3].stop+1:block1[3].stop)
    ])
    return filter(x->block_size(x) > 0, out)
end

function solve(filename::String, init_only::Bool)::Int64
    contiguous_on_ranges = Vector{Block}([])
    for comm in commands(filename, init_only=init_only)
        nblock = comm[2]
        # Replace all other blocks with difference of themselves and intersection with nblock
        nranges = Vector{Block}([])
        for r in contiguous_on_ranges
            int = block_intersect(r, nblock)
            if prod(length.(int)) > 0 # There is intersection
                append!(nranges, block_difference(r, int))
            else
                push!(nranges, r)
            end
        end
        contiguous_on_ranges = nranges
        # If on command, add nblock too
        if comm[1] # On
            push!(contiguous_on_ranges, nblock)
        end
    end
    out_sum = sum(block_size, contiguous_on_ranges)
    return out_sum
end

solve (generic function with 2 methods)

# Part1

In [60]:
println("Part 1 test: $(solve("test1.txt", true))")
println("Part 1 solution: $(solve("input.txt", true))")

Part 1 test: 590784
Part 1 solution: 556501


# Part 2

In [63]:
println("Part 2 test (initialization only): $(solve("test2.txt", true))")
println("Part 2 test: $(solve("test2.txt", false))")
@time println("Part 2 solution: $(solve("input.txt", false))")

Part 2 test (initialization only): 474140
Part 2 test: 2758514936282235
Part 2 solution: 1217140271559773
  0.458811 seconds (6.08 M allocations: 397.061 MiB, 4.39% gc time)
